<a href="https://colab.research.google.com/github/praveen-raj-m/Text-generation/blob/main/textgenDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np
import re
import tensorflow as tf

In [16]:
f = open("Alice.txt",'r')
data = f.read()

In [17]:
tokenizer = Tokenizer()
corpus = data.lower().split(".")
for i in range(0,len(corpus)):
  s = re.sub(' +',' ',(re.sub(r'[^\w]', ' ', corpus[i])))
  corpus[i] = s
print(len(corpus))
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

1223


In [18]:
input_sequences = []
for j in corpus:
 token_list = tokenizer.texts_to_sequences([j])[0]
 for i in range(1, len(token_list)):
  n_gram_sequence = token_list[:i+1]
  input_sequences.append(n_gram_sequence)

In [19]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [20]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [40]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 312, 100)          303800    
_________________________________________________________________
lstm_12 (LSTM)               (None, 312, 256)          365568    
_________________________________________________________________
dropout_11 (Dropout)         (None, 312, 256)          0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 3038)              780766    
Total params: 1,975,446
Trainable params: 1,975,446
Non-trainable params: 0
____________________________________________

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

filepath = "/content/drive/MyDrive/Text-Generator/model_training.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')
early_stop = EarlyStopping(monitor='accuracy',
                           patience=7,
                           restore_best_weights=True,
                           mode='max')
callbacks = [checkpoint,early_stop]

In [41]:
history = model.fit(predictors, label, epochs=100, verbose=1,callbacks=callbacks)

Epoch 1/100
921/921 [==============================] - 42s 43ms/step - loss: 6.4487 - accuracy: 0.0546

Epoch 00001: loss improved from inf to 6.21541, saving model to /content/drive/MyDrive/Text-Generator/model_training.hdf5
Epoch 2/100
921/921 [==============================] - 40s 43ms/step - loss: 5.8156 - accuracy: 0.0708

Epoch 00002: loss improved from 6.21541 to 5.79415, saving model to /content/drive/MyDrive/Text-Generator/model_training.hdf5
Epoch 3/100
921/921 [==============================] - 40s 43ms/step - loss: 5.4987 - accuracy: 0.0940

Epoch 00003: loss improved from 5.79415 to 5.46979, saving model to /content/drive/MyDrive/Text-Generator/model_training.hdf5
Epoch 4/100
921/921 [==============================] - 40s 43ms/step - loss: 5.2235 - accuracy: 0.1195

Epoch 00004: loss improved from 5.46979 to 5.21181, saving model to /content/drive/MyDrive/Text-Generator/model_training.hdf5
Epoch 5/100
921/921 [==============================] - 40s 43ms/step - loss: 4.9804 

In [45]:
model.save("/content/drive/MyDrive/Text-Generator/model2.hdf5")

In [46]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/Text-Generator/model2.hdf5')


In [49]:
seed_text = "we indeed cried"
next_words = 10
  
for _ in range(next_words):
 token_list = tokenizer.texts_to_sequences([seed_text])[0]
 token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
 predicted = np.argmax(model.predict(token_list), axis=-1)
 output_word = ""
 for word, index in tokenizer.word_index.items():
  if index == predicted:
   output_word = word
   break
 seed_text += " " + output_word
print(seed_text)

we indeed cried the mouse who was trembling down to the end of
